In [ ]:
# sample using an Estimator model

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np

# load mnist dataset
((train_data, train_labels), (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)  # not required

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)  # not required

# Create the Estimator
def make_classifier(model_dir=None):
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()
    
    def cnn_model_fn(features, labels, mode):
      import tensorflow.compat.v1 as tf
      tf.disable_v2_behavior()
    
      import numpy as np
      """Model function for CNN."""
      # Input Layer
      if isinstance(features, dict):
        features = features['x']
      input_layer = tf.reshape(features, [-1, 28, 28, 1])
        

      # Convolutional Layer #1
      conv1 = tf.layers.conv2d(
          inputs=input_layer,
          filters=32,
          kernel_size=[5, 5],
          padding="same",
          activation=tf.nn.relu)

      # Pooling Layer #1
      pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

      # Convolutional Layer #2 and Pooling Layer #2
      conv2 = tf.layers.conv2d(
          inputs=pool1,
          filters=64,
          kernel_size=[5, 5],
          padding="same",
          activation=tf.nn.relu)
      pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

      # Dense Layer
      pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
      dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
      dropout = tf.layers.dropout(
          inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

      # Logits Layer
      logits = tf.layers.dense(inputs=dropout, units=10)

      predictions = {
          # Generate predictions (for PREDICT and EVAL mode)
          "classes": tf.argmax(input=logits, axis=1),
          # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
          # `logging_hook`.
          "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
      }

      if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

      # Calculate Loss (for both TRAIN and EVAL modes)
      loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

      # Configure the Training Op (for TRAIN mode)
      if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

      # Add evaluation metrics (for EVAL mode)
      eval_metric_ops = {
          "accuracy": tf.metrics.accuracy(
              labels=labels, predictions=predictions["classes"])
      }
      return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    
    mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir=model_dir)
    return mnist_classifier
    
mnist_classifier = make_classifier(model_dir="/tmp/mnist_convnet_model")
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

# train one step and display the probabilties
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=1,
    hooks=[logging_hook])
# evaluate
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

In [ ]:
!ls /tmp/mnist_convnet_model

In [ ]:
!pip freeze | grep dill

In [ ]:
import omegaml as om 

om.models.get('tf-model-mnist-fn')

In [ ]:
# we can save the model so it can be reused and re-fitted on the cluster

import omegaml as om 
from omegaml.backends.tensorflow import TFEstimatorModel

om.datasets.put(train_data, 'mnist-X')
om.datasets.put(train_labels, 'mnist-Y')

saveable_model = TFEstimatorModel(estimator_fn=make_classifier, model=mnist_classifier, v1_compat=True)
om.models.put(saveable_model, 'tf-model-mnist-fn')


In [ ]:
# fit on the cluster. this may take a while
om.runtime.model('tf-model-mnist-fn').fit('mnist-X', 'mnist-Y').get()

In [ ]:
# re-load the model and predict from a numpy array
train_data = om.datasets.get('mnist-X')

X = train_data[0:2, :]

model_ = om.models.get('tf-model-mnist-fn')
[v for v in model_.predict(X)]

In [ ]:
# predict on the cluster via direct input as numpy, or the numpy array stored as a dataset
train_data = om.datasets.get('mnist-X')

#we can save the data as stacked images
om.datasets.put(train_data[0:2, :], 'mnist-sample')
om.runtime.model('tf-model-mnist-fn').predict('mnist-sample').get()
# or just pass the data as is
om.runtime.model('tf-model-mnist-fn').predict(train_data[0:2, :]).get()

In [ ]:
# we can save the model as a SavedModel using a standard or custom input fn
# ServingInput is a helper to create a corresponding input fn from a given like=dataset

from omegaml.backends.tensorflow.tfsavedmodel import ServingInput

input_fn = ServingInput(features=['x'], like=train_data.reshape((-1, 28, 28, 1)), v1_compat=True)

om.models.put(mnist_classifier, 'tf-model-mnist-estimator', 
              serving_input_fn=input_fn)

In [ ]:
# once saved we can again use the runtime to predict
om.datasets.put(train_data[0:2, :].reshape((-1, 28, 28, 1)), 'mnist-sample')
om.runtime.model('tf-model-mnist-estimator').predict('mnist-sample').get()

In [ ]:
# or use the model REST API 
import requests
from omegaml.client.auth import OmegaRestApiAuth

url = getattr(om.defaults, 'OMEGA_RESTAPI_URL', 'http://localhost:5000')
auth = OmegaRestApiAuth.make_from(om)
dataset = 'mnist-sample'
modelname = 'tf-model-mnist-estimator'
predict_url = '{url}/api/v1/model/{modelname}/predict?datax={dataset}'.format(**locals())
resp = requests.put(predict_url, auth=auth)
print(predict_url)
print(resp.json())